In [3]:
import sqlite3
netflix_db = sqlite3.connect('netflix.sqlite')
task_db = sqlite3.connect("task1.sqlite")
netflix_db.row_factory = lambda cursor, row: {col[0]: row[i] for i, col in enumerate(cursor.description)}

Создаём таблицу:

In [4]:
task_cur = task_db.cursor()
with task_db:
  task_cur.execute("""
  CREATE TABLE films (
    film_id INTEGER PRIMARY KEY AUTOINCREMENT,
    type TEXT,
    title TEXT,
    director TEXT,
    country TEXT,
    date_added TEXT,
    release_year INTEGER,
    rating TEXT,
    duration TEXT,
    listed_in TEXT,
    description TEXT
  );
  """)
  task_cur.execute("""
  CREATE TABLE actors (
    actor_id INTEGER PRIMARY KEY AUTOINCREMENT,
    actor_name TEXT UNIQUE
  );
  """)
  task_cur.execute("""
    CREATE TABLE actors_films (
    film_id INTEGER,
    actor_id INTEGER,
    PRIMARY KEY (actor_id, film_id),
    FOREIGN KEY (film_id) REFERENCES films(film_id),
    FOREIGN KEY (actor_id) REFERENCES actors(actor_id)
  );
  """)

Задание 1. Нормализуем базу данных относительно актеров. 

In [5]:
netflix_cur = netflix_db.cursor()
task_cur = task_db.cursor()

insert_query = """
INSERT INTO films (type, title, director, country, date_added, release_year, rating, duration, listed_in, description)
VALUES (:type, :title, :director, :country, :date_added, :release_year, :rating, :duration, :listed_in, :description);
"""

actors_films_insert_query = """
INSERT OR IGNORE INTO actors_films (film_id, actor_id)
VALUES (?, (SELECT actor_id FROM actors WHERE actor_name = ?));
"""

with netflix_db, task_db:
    for row in netflix_cur.execute("SELECT * FROM netflix_titles"):
        task_cur.execute(insert_query, row)
        film_id = task_cur.execute("SELECT last_insert_rowid() FROM films;").fetchone()[0]
        
        for actor in row["cast"].split(", "):
            task_cur.execute("INSERT OR IGNORE INTO actors (actor_name) VALUES (?)", (actor,))
            task_cur.execute(actors_films_insert_query, (film_id, actor))

Задание 2. Вычисляем наиболее часто работающую друг с другом пару актеров.

In [6]:
with task_db:
    most_freq_pair = task_cur.execute("""
    SELECT a1.actor_name as name1, a2.actor_name as name2
    FROM actors_films af1
    JOIN actors_films af2 ON af1.film_id = af2.film_id AND af1.actor_id < af2.actor_id
    JOIN actors a1 ON af1.actor_id = a1.actor_id
    JOIN actors a2 ON af2.actor_id = a2.actor_id
    GROUP BY af1.actor_id, af2.actor_id
    ORDER BY COUNT(*) DESC
    LIMIT 1;
    """).fetchone()
most_freq_pair

('John Paul Tremblay', 'Robb Wells')